# Imports

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [8]:
%load_ext autoreload
%autoreload 2

from load_data import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load Data

# Analysis